In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import pandas as pd
import numpy as np
import requests
import json
import sys
import os
import pickle
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [23]:
!pip install -e ift6758

Obtaining file:///Users/krishnamaneeshad/milestone_3/ift6758/docker-project-template-main/ift6758
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ift6758
    Found existing installation: ift6758 0.1.0
    Uninstalling ift6758-0.1.0:
      Successfully uninstalled ift6758-0.1.0
  Running setup.py develop for ift6758


In [24]:

from ift6758.client.game_client import GameClient as game_client
from ift6758.client.serving_client import ServingClient as serving_client

In [25]:
model_names = ['lrd', 'lrda', 'lda', 'xgboost', 'xgboost-tuning']
workspace_names = ['kleitoun', '']
version_list = ['1.0.0','']

In [31]:
sc = serving_client(ip = "serving", port = "5000")

In [ ]:
sc.logs()

In [27]:
########################################################## DOWNLOAD MODEL FUNCTIONALITY ######################################

workspace = widgets.Dropdown(
    description='Workspace',
    value=workspace_names[0],
    options=workspace_names

)
model_name = widgets.Dropdown(
    description='Model Name',
    value=model_names[0],
    options=model_names

)
version = widgets.Dropdown(
    description='Model Version',
    value=version_list[0],
    options=version_list

)

download_button = widgets.Button(description="Download Model")

output = widgets.Output()


@output.capture()
def on_button_clicked(b):
    sc.download_registry_model(workspace = workspace.value, model = model_name.value, version = str(version.value))
    with output:
        print("Downloaded")
    
    
container2 = widgets.HBox(children=[workspace,model_name,version])
display(container2)
display(download_button)
download_button.on_click(on_button_clicked)
display(output)

########################################################## PING GAME FUNCTIONALITY ######################################


game_id_input = widgets.Text(description='Game ID')
display(game_id_input)
button = widgets.Button(description="Ping Game")
display(button)



output = widgets.Output()
gc = game_client()


@output.capture()
def on_button_clicked(b):
    gc.get_game(game_id = game_id_input.value)
    model_df, idx, current_status_df = gc.ping_game(sc.model)
    print("*********")
    print(model_df.columns)
    preds = sc.predict(model_df)
    preds = [pred for pred in preds]
    
    current_status_df["predictions"] = preds
    current_status_df["xG"] = current_status_df["predictions"].apply(lambda x: 1 if(x>0.50) else 0)

    
    home_team = gc.home_team
    away_team = gc.away_team
    grouped_df = current_status_df.groupby("shot_by_team")["xG"].agg("sum")
    dashboard_dict = {f"Period: {gc.dashboard_period}": ["Home Team",home_team, grouped_df["Home"]], f"Time Remaining: {gc.dashboard_time}": ["Away Team",away_team, grouped_df["Away"]]}
    dashboard_df = pd.DataFrame(dashboard_dict)
    with output:
        
        #define figure and axes
        fig, ax = plt.subplots()

        #hide the axes
        fig.patch.set_visible(False)
        ax.axis('off')
        ax.axis('tight')

        #create table
        table = ax.table(cellText=dashboard_df.values, colLabels=dashboard_df.columns, loc='center')

        #display table
        fig.tight_layout()
        plt.show()
        display(current_status_df)
    

button.on_click(on_button_clicked)
display(output)

Button(description='Download Model', style=ButtonStyle())

Output()

Text(value='', description='Game ID')

Button(description='Ping Game', style=ButtonStyle())

Output()

In [28]:
sc.logs()

b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>500 Internal Server Error</title>\n<h1>Internal Server Error</h1>\n<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>\n'


<Response [500]>